In [263]:
import lxml
from datetime import datetime
import os
import glob
from pathlib import Path
import polars as pl
import re
from lxml import etree, objectify

In [81]:
PATH_DIR = "data/2022/"

In [85]:
files = list(Path(PATH_DIR).rglob("*.xml"))
files

[PosixPath('data/2022/668566_T.xml'),
 PosixPath('data/2022/837018_T.xml'),
 PosixPath('data/2022/2029398_T.xml'),
 PosixPath('data/2022/1997568_T.xml'),
 PosixPath('data/2022/2038636_T.xml'),
 PosixPath('data/2022/623983_T.xml'),
 PosixPath('data/2022/2022844_T.xml'),
 PosixPath('data/2022/1333270_T.xml'),
 PosixPath('data/2022/913270_T.xml'),
 PosixPath('data/2022/1825166_T.xml'),
 PosixPath('data/2022/1821369_T.xml'),
 PosixPath('data/2022/1131204_T.xml'),
 PosixPath('data/2022/2024576_T.xml'),
 PosixPath('data/2022/1357711_T.xml'),
 PosixPath('data/2022/1186205_T.xml'),
 PosixPath('data/2022/1233867_T.xml'),
 PosixPath('data/2022/988550_T.xml'),
 PosixPath('data/2022/1205525_T.xml'),
 PosixPath('data/2022/1205560_T.xml'),
 PosixPath('data/2022/998918_T.xml'),
 PosixPath('data/2022/1845966_T.xml'),
 PosixPath('data/2022/1522399_T.xml'),
 PosixPath('data/2022/1647262_T.xml'),
 PosixPath('data/2022/2082710_T.xml'),
 PosixPath('data/2022/1107388_T.xml'),
 PosixPath('data/2022/828623_T.

In [659]:
events = []
for file in files:
    event = {}
    event['file'] = Path(file).stem
    event['year_upload'] = int(Path(file).parts[1])
    # print(file)
    for _, elem in etree.iterparse(file):
        tag = elem.tag
        if tag is not None:
            text = elem.text
            if tag == 'Body':
                event['body_orig'] = text
                
                body = extract_info_from_conference_call_body(text)
                event['corp_participants'] = body['corp_participants']
                event['corp_participants_colapsed'] = body['corp_participants_colapsed']
                event['conf_participants'] = body['conf_participants']
                event['conf_participants_colapsed'] = body['conf_participants_colapsed']
                event['presentation'] = body['presentation']
                event['qa'] = body['qa']
                
            if tag == 'EventStory':
                event['action'] = elem.attrib['action']
                event['story_type'] = elem.attrib['storyType']
                event['version'] = elem.attrib['version']
            if tag == 'eventTitle':
                event['event_title'] = text
            if tag == 'city':
                event['city'] = text
            if tag == 'companyName':
                event['company_name'] = text
            if tag == 'companyTicker':
                event['company_ticker'] = text
            if tag == 'startDate':
                event['date'] = datetime.strptime(text, '%d-%b-%y %I:%M%p %Z')
            if tag == 'Event':
                event['id'] = int(elem.attrib['Id'])
                event['last_update'] = datetime.strptime(elem.attrib['lastUpdate'], '%A, %B %d, %Y at %I:%M:%S%p %Z')
                event['event_type_id'] = int(elem.attrib['eventTypeId'])
                event['event_type_name'] = elem.attrib['eventTypeName']
    events.append(event)

In [688]:
events[0].keys()

dict_keys(['file', 'year_upload', 'body_orig', 'corp_participants', 'conf_participants', 'presentation', 'qa', 'action', 'story_type', 'version', 'event_title', 'city', 'company_name', 'company_ticker', 'date', 'id', 'last_update', 'event_type_id', 'event_type_name'])

In [687]:
len(['file', 'year_upload', 'id', 'last_update', 'event_type_id', 'event_type_name', 'action', 'story_type', 'version', 'event_title', 'city', 'company_name', 'company_ticker', 'date', 'body_orig', 'corp_participants', 'conf_participants'])

17

In [684]:
new_dict = [{key: events[i][key] for key in events[i].keys() if key in ['file', 'year_upload', 'id', 'last_update', 'event_type_id', 'event_type_name', 'action', 'story_type', 'version', 'event_title', 'city', 'company_name', 'company_ticker', 'date', 'body_orig', 'corp_participants', 'conf_participants']}
            for i in range(len(events))]

In [685]:
eventsDF = pl.from_dicts(new_dict)
eventsDF

file,year_upload,body_orig,corp_participants,conf_participants,action,story_type,version,event_title,city,company_name,company_ticker,date,id,last_update,event_type_id,event_type_name
str,i64,str,list[list[str]],list[list[str]],str,str,str,str,str,str,str,datetime[μs],i64,datetime[μs],i64,str
"""668566_T""",2022,"""Q3 2002 Jeffer…","[[""Richard Handler"", ""Jefferies Group, Inc. - Chairman and CEO""], [""Joseph Schenk"", ""Jefferies Group, Inc. - CFO""], [""John Shaw, Jr."", ""Jefferies Group, Inc. - Pres, COO, Director""]]","[[""Lauren Smith"", ""KBW""], [""Colin Clark"", ""Salomon Smith Barney""]]","""publish""","""transcript""","""Final""","""Q3 2002 Jeffer…","""NEW YORK""","""Jefferies Grou…","""JEF.N^C13""",2002-10-22 18:00:00,668566,2022-11-02 17:25:11,1,"""Earning Confer…"
"""837018_T""",2022,"""Home Depot Inc…","[[""Diane Dayhoff"", ""The Home Depot, Inc. - Vice President of Investor Relations""], [""Bob L. Nardelli"", ""The Home Depot, Inc. - Chairman, President and CEO""], … [""John Costello"", ""The Home Depot, Inc. - Merchandising &amp; Marketing""]]","[[""Eran"", ""- Analyst""], [""Dan Yau"", ""- Analyst""], … [""Greg"", ""- Analyst""]]","""publish""","""transcript""","""Final""","""Home Depot Inc…","""ATLANTA""","""Home Depot Inc…","""HD.N""",2004-01-16 14:00:00,837018,2022-12-06 14:31:50,31,"""Analyst Meetin…"
"""2029398_T""",2022,"""Q3 2008 Canadi…","[[""John Robertson"", ""The Ruth Group - IR Contact""], [""Dr. Shawn Qu"", ""Canadian Solar, Inc. - Chairman, CEO""], [""Arthur Chien"", ""Canadian Solar, Inc. - CFO""]]","[[""Steve O'Rourke"", ""Deutsche Bank - Analyst""], [""Sanjay Shrestha"", ""Lazard Capital Markets - Analyst""], … [""Paul Leming"", ""Soleil Securities - Analyst""]]","""publish""","""transcript""","""Final""","""Q3 2008 Canadi…","""KITCHENER""","""Canadian Solar…",""" """,2008-11-21 13:00:00,2029398,2022-08-18 22:56:37,1,"""Earning Confer…"
"""1997568_T""",2022,"""Q3 2008 Bankin…","[[""Gloria Ortiz"", ""Bankinter S.A. - CFO""], [""Jaime Hernandez"", ""Bankinter S.A. - IR""]]",[],"""publish""","""transcript""","""Final""","""Q3 2008 Bankin…","""Madrid""","""Bankinter SA""","""BKT.MC""",2008-10-16 07:00:00,1997568,2022-11-30 08:57:05,1,"""Earning Confer…"
"""2038636_T""",2022,"""Noble Corporat…","[[""David Williams"", ""Noble Corporation - President, CEO""]]","[[""Rob MacKenzie"", ""FBR - Analyst""]]","""publish""","""transcript""","""Final""","""Noble Corporat…","""new York""","""Noble Holding …",""" """,2008-12-03 14:15:00,2038636,2022-10-19 14:45:01,7,"""Conference Pre…"
"""623983_T""",2022,"""Q1 2002 Health…","[[""David Emery"", ""Healthcare Realty Trust - Chairman and CEO""], [""Mancini (ph)"", ""Healthcare Realty Trust""], [""Timothy Wallace"", ""Healthcare Realty Trust Incorporated - EVP and CFO""]]","[[""Stephen Swett (ph)"", ""Wacovia Securities""], [""Jerry Doctrow (ph)"", ""Liche Mason""], … [""Robert Belzer (ph)"", ""Prudential Securities""]]","""publish""","""transcript""","""Final""","""Q1 2002 Health…","""NASHVILLE""","""Healthcare Rea…","""HR.N^G22""",2002-04-26 14:00:00,623983,2022-07-22 13:21:51,1,"""Earning Confer…"
"""2022844_T""",2022,"""Q3 2008 Uraniu…","[[""Jean Nortier"", ""Uranium One Inc. - Pres. and CEO""], [""Steve Magnuson"", ""Uranium One Inc. - COO""], [""Robin Merrifield"", ""Uranium One Inc. - CFO""]]","[[""Duncan McKeen"", ""Macquarie Capital Markets - Analyst""], [""Adam Schatzker"", ""RBC Capital Markets - Analyst""], … [""David Snow"", ""Energy Equities - Analyst""]]","""publish""","""transcript""","""Final""","""Q3 2008 Uraniu…","""VANCOUVER""","""Uranium One In…","""UUU.TO^J13""",2008-11-14 15:00:00,2022844,2022-10-27 12:59:09,1,"""Earning Confer…"
"""1333270_T""",2022,"""Cardiac Scienc…","[[""John Hinson"", ""Cardiac Science Corporation - President, CEO and Director""]]",[],"""publish""","""transcript""","""Final""","""Cardiac Scienc…","""New York""","""CS Estate Inc""","""CSCX.OQ^L10""",2006-06-15 13:30:00,1333270,2022-11-30 10:49:05,7,"""Conference Pre…"


In [652]:
body = eventsDF.filter(pl.col("file")=="618360_T").select("body").item()

In [653]:
print(body)

Q1 2002 Noble Drilling Earnings Conference Call

United Kingdom Oct 19, 2022 (Thomson StreetEvents) -- Edited Transcript of Noble Holding Corporation PLC earnings conference call or presentation Friday, April 26, 2002 at 3:00:00pm GMT

TEXT version of Transcript

Transcript
--------------------------------------------------------------------------------
Operator   [1]
--------------------------------------------------------------------------------
 Welcome to the Noble Drilling Corporation first quarter earnings conference call. During the presentations, all participants will be in the listen-only mode. Afterwards we'll conduct a question and answer session. At that time, if you have a question, press the 1 followed by the 4 on your telephone. This conference call is being recorded Friday April 26, 2002. I would like to turn the conference over to Mr. James Day, Chairman and CEO of Noble Drilling Corporation. Please go ahead, sir.
-------------------------------------------------------

In [654]:
def seperate_conference_call(string):
    start_word_corp_participants = "Corporate Participants\r\n================================================================================\r\n"
    end_word_corp_participants = "================================================================================"

    start_word_conf_participants = "Conference Call Participants\r\n================================================================================\r\n"
    end_word_conf_participants = "================================================================================"

    # es kann auch der Fall sein, dass anstatt von Presentation "Transcript" steht
    start_word_presentation = "Presentation\r\n--------------------------------------------------------------------------------"
    end_word_presentation = "================================================================================"
    start_word_transcript = "Transcript\r\n--------------------------------------------------------------------------------"
    end_word_transcript = "================================================================================"

    start_word_qa = "Questions and Answers\r\n--------------------------------------------------------------------------------"
    end_word_qa = "================================================================================"

    if start_word_corp_participants in string:
        start_index_corp_participants = string.find(start_word_corp_participants) + len(start_word_corp_participants)
        end_index_corp_participants = string.find(end_word_corp_participants, start_index_corp_participants)
        corp_participants = string[start_index_corp_participants:end_index_corp_participants]
    else:
        corp_participants = ""
    
    if start_word_conf_participants in string:
        start_index_conf_participants = string.find(start_word_conf_participants) + len(start_word_conf_participants)
        end_index_conf_participants = string.find(end_word_conf_participants, start_index_conf_participants)
        conf_participants = string[start_index_conf_participants:end_index_conf_participants]
    else:
        conf_participants = ""
    
    if start_word_presentation in string:
        start_index_presentation = string.find(start_word_presentation) + len(start_word_presentation)
        end_index_presentation = string.find(end_word_presentation, start_index_presentation)
        presentation = string[start_index_presentation:end_index_presentation]
    elif start_word_transcript in string:
        start_index_presentation = string.find(start_word_transcript) + len(start_word_transcript)
        end_index_presentation = string.find(end_word_presentation, start_index_presentation)
        presentation = string[start_index_presentation:end_index_presentation]
    else:
        presentation = ""
    
    if start_word_qa in string:
        start_index_qa = string.find(start_word_qa) + len(start_word_qa)
        end_index_qa = string.find(end_word_qa, start_index_qa)
        qa = string[start_index_qa:end_index_qa]
    else:
        qa = ""
    
    output = {'corp_participants': corp_participants,
              'conf_participants': conf_participants,
              'presentation': presentation,
              'qa': qa}
    
    return output

In [655]:
conference_call_sep_raw = seperate_conference_call(body)

In [656]:
conference_call_sep_raw

{'corp_participants': '',
 'conf_participants': '',
 'presentation': "\r\nOperator   [1]\r\n--------------------------------------------------------------------------------\r\n Welcome to the Noble Drilling Corporation first quarter earnings conference call. During the presentations, all participants will be in the listen-only mode. Afterwards we'll conduct a question and answer session. At that time, if you have a question, press the 1 followed by the 4 on your telephone. This conference call is being recorded Friday April 26, 2002. I would like to turn the conference over to Mr. James Day, Chairman and CEO of Noble Drilling Corporation. Please go ahead, sir.\r\n--------------------------------------------------------------------------------\r\n James Day,    [2]\r\n--------------------------------------------------------------------------------\r\n Thank you. Good morning, everyone. With me this morning is Bob Campbell, President of the company, Mark Jackson, Chief Financial Officer,

In [657]:
## Participants
# Corporation Participants
corp_participants = conference_call_sep_raw['corp_participants'].split('*')
corp_participants = [[el.strip() for el in pair.split('\r\n') if el.strip()] for pair in corp_participants if pair.strip()]
corp_participants_colapsed = [',  '.join(pair) for pair in corp_participants]

# Conference Call Participants
conf_participants = conference_call_sep_raw['conf_participants'].split('*')
conf_participants = [[el.strip() for el in pair.split('\r\n') if el.strip()] for pair in conf_participants if pair.strip()]
conf_participants_colapsed = [',  '.join(pair) for pair in conf_participants]

In [658]:
## Presentations
# Split presentation into slides
split_presentation = '--------------------------------------------------------------------------------\r\n'
presentation = conference_call_sep_raw['presentation'].split(split_presentation)
presentation = [el.strip() for el in presentation] # if el.strip()
# following removes the double spaces between speaker and his position
# presentation = [re.sub(' +', ' ', el.strip()) for el in presentation if el.strip()]

# Split presentation into speakers and texts
presentation_speakers = [el for el in presentation if re.match('.+\s{2,}\[\d+\]', el) or (re.match('\[\d+\]', el) and len(el)<=5)] # if the speaker is not mentioned
presentation_texts = [el for el in presentation if el not in presentation_speakers]



# Note: if no speaker or no text is found, the presentation is not included
if len(presentation_speakers)==0 or len(presentation_texts)==0:
  print("Warning: presentation_speakers or presentation_texts is empty")

regex_pattern = r"(.*)\s{2,}\[(\d+)\]$"
presentation_speakers_ordered = [[int(re.search(regex_pattern, el).group(2)), re.search(regex_pattern, el).group(1).strip()] if not re.match('\[\d+\]', el) 
                                 else [int(re.search(r'\[(\d+)\]', el).group(1)), "unknown"] # if the speaker is not mentioned
                                 for el in presentation_speakers]

presentation_speakers_ordered = [[el[0],
  el[1],
  "cooperation" if el[1] in corp_participants_colapsed
  else "conference" if el[1] in conf_participants_colapsed
  else el[1]]
 for el in presentation_speakers_ordered]


if len(presentation_speakers) != len(presentation_texts):
    print('Warning: presentation_speakers and presentation_texts have different lengths')
    if len(presentation_speakers) > len(presentation_texts):
      missing = len(presentation_speakers) - len(presentation_texts)
      presentation_texts = presentation_texts + [''] * missing
    if len(presentation_speakers) < len(presentation_texts):
      missing = len(presentation_texts) - len(presentation_speakers)
      last_speaker = presentation_speakers_ordered[-1][0]
      for i in range(missing):
        presentation_speakers.append([last_speaker+i, "unknown", "unknown"])
        
presentation_ordered = [[presentation_speakers_ordered[i][0], presentation_speakers_ordered[i][1], presentation_speakers_ordered[i][2], presentation_texts[i]] for i in range(len(presentation_speakers))]
presentation_ordered

[[1,
  'Operator',
  'Operator',
  "Welcome to the Noble Drilling Corporation first quarter earnings conference call. During the presentations, all participants will be in the listen-only mode. Afterwards we'll conduct a question and answer session. At that time, if you have a question, press the 1 followed by the 4 on your telephone. This conference call is being recorded Friday April 26, 2002. I would like to turn the conference over to Mr. James Day, Chairman and CEO of Noble Drilling Corporation. Please go ahead, sir."],
 [2,
  'James Day,',
  'James Day,',
  "Thank you. Good morning, everyone. With me this morning is Bob Campbell, President of the company, Mark Jackson, Chief Financial Officer, and Julie Robertson, Senior Vice President of Administration. We will go through our normal drill, that is, I'll ask Mark to make comments about the financial results, which I might add given the market, we're pleased of the results. And I'll turn the microphone over to Bob Campbell, and Bo

In [624]:
presentation

['Operator   [1]',
 "Welcome to the Noble Drilling Corporation first quarter earnings conference call. During the presentations, all participants will be in the listen-only mode. Afterwards we'll conduct a question and answer session. At that time, if you have a question, press the 1 followed by the 4 on your telephone. This conference call is being recorded Friday April 26, 2002. I would like to turn the conference over to Mr. James Day, Chairman and CEO of Noble Drilling Corporation. Please go ahead, sir.",
 'James Day,    [2]',
 "Thank you. Good morning, everyone. With me this morning is Bob Campbell, President of the company, Mark Jackson, Chief Financial Officer, and Julie Robertson, Senior Vice President of Administration. We will go through our normal drill, that is, I'll ask Mark to make comments about the financial results, which I might add given the market, we're pleased of the results. And I'll turn the microphone over to Bob Campbell, and Bob can talk about the five issues

In [622]:
presentation

['Operator   [1]',
 "Welcome to the Noble Drilling Corporation first quarter earnings conference call. During the presentations, all participants will be in the listen-only mode. Afterwards we'll conduct a question and answer session. At that time, if you have a question, press the 1 followed by the 4 on your telephone. This conference call is being recorded Friday April 26, 2002. I would like to turn the conference over to Mr. James Day, Chairman and CEO of Noble Drilling Corporation. Please go ahead, sir.",
 'James Day,    [2]',
 "Thank you. Good morning, everyone. With me this morning is Bob Campbell, President of the company, Mark Jackson, Chief Financial Officer, and Julie Robertson, Senior Vice President of Administration. We will go through our normal drill, that is, I'll ask Mark to make comments about the financial results, which I might add given the market, we're pleased of the results. And I'll turn the microphone over to Bob Campbell, and Bob can talk about the five issues

In [618]:
presentation_speakers

['Operator   [1]',
 'James Day,    [2]',
 'Mark Jackson,    [3]',
 'Bob Campbell,    [4]',
 'James Day,    [5]',
 'Operator   [6]',
 'Scott Gill,    [7]',
 'James Day,    [8]',
 'Scott Gill,    [9]',
 'James Day,    [10]',
 'Mark Jackson,    [11]',
 'Scott Gill,    [12]',
 'Mark Jackson,    [13]',
 'Scott Gill,    [14]',
 'James Day,    [15]',
 'Scott Gill,    [16]',
 'James Day,    [17]',
 'Scott Gill,    [18]',
 'James Day,    [19]',
 'Scott Gill,    [20]',
 'James Day,    [21]',
 'Operator   [22]',
 'KEN FIELD,    [23]',
 'James Day,    [24]',
 'KEN FIELD,    [25]',
 'James Day,    [26]',
 'KEN FIELD,    [27]',
 'James Day,    [28]',
 'KEN FIELD,    [29]',
 'James Day,    [30]',
 'KEN FIELD,    [31]',
 'James Day,    [32]',
 'Operator   [33]',
 'Bill Sanchez,    [34]',
 'James Day,    [35]',
 'Bill Sanchez,    [36]',
 'James Day,    [37]',
 'Bill Sanchez,    [38]',
 'James Day,    [39]',
 'Bill Sanchez,    [40]',
 'James Day,    [41]',
 'Bill Sanchez,    [42]',
 'James Day,    [43]'

In [473]:
conference_call_sep_raw['qa']

"\r\nOperator   [1]\r\n--------------------------------------------------------------------------------\r\n Thank you. Ladies and gentlemen, we will now conduct a question and answer session. (Operator Instructions). One moment for your first question. Your first question comes from Duncan McKeen of Macquarie Capital. Please go ahead, sir. Mr. McKeen?\r\n--------------------------------------------------------------------------------\r\n Duncan McKeen,  Macquarie Capital Markets - Analyst   [2]\r\n--------------------------------------------------------------------------------\r\n Sorry about that, operator, I'm here. Good morning. I missed what you had said on the cash cost guidance for South Inkai. I was wondering if you could just go through that again.\r\n--------------------------------------------------------------------------------\r\n Jean Nortier,  Uranium One Inc. - Pres. and CEO   [3]\r\n--------------------------------------------------------------------------------\r\n Yes

In [645]:
def extract_info_from_conference_call_part(part, corp_participants, conf_participants, type='presentation'):
    # Part is either presentation or qa
    # Split presentation into slides
    split_symbol = '--------------------------------------------------------------------------------\r\n'
    part_split = part.split(split_symbol)
    part_split = [el.strip() for el in part_split] #  if el.strip()
    # removes the double spaces between speaker and his position
    # presentation = [re.sub(' +', ' ', el.strip()) for el in presentation if el.strip()]

    # Split presentation into speakers and texts
    speakers = [el for el in part_split if re.match('.+\s{2,}\[\d+\]', el) or (re.match('\[\d+\]', el) and len(el)<=5)]
    texts = [el for el in part_split if el not in speakers]

    n_speakers = len(speakers)
    n_texts = len(texts)
    
    # Note: if no speaker or no text is found, the presentation is not included
    if n_speakers==0:
        print ("Warning: No speakers present at " + type)
        return None
    if n_texts==0:
        print("Warning: No texts present " + type)
        return None
    
    regex_pattern = r"(.*)\s{2,}\[(\d+)\]$"
    speakers_ordered = [[int(re.search(regex_pattern, el).group(2)), re.search(regex_pattern, el).group(1).strip()] if not re.match('\[\d+\]', el) 
                                 else [int(re.search(r'\[(\d+)\]', el).group(1)), "unknown"] # if the speaker is not mentioned
                                 for el in speakers]

    speakers_ordered = [[el[0],
    el[1],
    "operator" if el[1] == "Operator"
    else "editor" if el[1] == "Editor"
    else "cooperation" if el[1] in corp_participants
    else "conference" if el[1] in conf_participants
    else el[1] if corp_participants != [] and conf_participants != []
    else "unknown"]
    for el in speakers_ordered]


    if len(speakers) != len(texts):
        print("Warning: presentation_speakers (", n_speakers,") and presentation_texts (", n_texts ,") have different lengths", sep="")
        if n_speakers > n_texts:
            missing = n_speakers - n_texts
            texts = texts + [''] * missing
            print("Warning: presentation_texts was extended with empty strings")
        if n_speakers < n_texts:
            missing = n_texts - n_speakers
            last_speaker = speakers_ordered[-1][0]
            for i in range(missing):
                speakers.append([last_speaker+i, "unknown", "unknown"])
            print("Warning: presentation_speakers was extended with unknown speakers")
    part_ordered = [[speakers_ordered[i][0],
                     speakers_ordered[i][1],
                     speakers_ordered[i][2],
                     texts[i]]
                    for i in range(len(speakers))]
    return part_ordered

In [638]:
extract_info_from_conference_call_part(conference_call_sep_raw['presentation'], corp_participants_colapsed, conf_participants_colapsed)

[[1,
  'Operator',
  'operator',
  "Welcome to the Noble Drilling Corporation first quarter earnings conference call. During the presentations, all participants will be in the listen-only mode. Afterwards we'll conduct a question and answer session. At that time, if you have a question, press the 1 followed by the 4 on your telephone. This conference call is being recorded Friday April 26, 2002. I would like to turn the conference over to Mr. James Day, Chairman and CEO of Noble Drilling Corporation. Please go ahead, sir."],
 [2,
  'James Day,',
  'unknown',
  "Thank you. Good morning, everyone. With me this morning is Bob Campbell, President of the company, Mark Jackson, Chief Financial Officer, and Julie Robertson, Senior Vice President of Administration. We will go through our normal drill, that is, I'll ask Mark to make comments about the financial results, which I might add given the market, we're pleased of the results. And I'll turn the microphone over to Bob Campbell, and Bob c

In [629]:
def extract_info_from_conference_call_sep(conference_call_sep_raw):
    ## Participants
    # Corporation Participants
    corp_participants = conference_call_sep_raw['corp_participants'].split('*')
    corp_participants = [[el.strip() for el in pair.split('\r\n') if el.strip()] for pair in corp_participants if pair.strip()]
    corp_participants_colapsed = [',  '.join(pair) for pair in corp_participants]

    # Conference Call Participants
    conf_participants = conference_call_sep_raw['conf_participants'].split('*')
    conf_participants = [[el.strip() for el in pair.split('\r\n') if el.strip()] for pair in conf_participants if pair.strip()]
    conf_participants_colapsed = [',  '.join(pair) for pair in conf_participants]
    
    presentation = extract_info_from_conference_call_part(conference_call_sep_raw['presentation'], corp_participants_colapsed, conf_participants_colapsed, type='presentation')
    qa = extract_info_from_conference_call_part(conference_call_sep_raw['qa'], corp_participants_colapsed, conf_participants_colapsed, type='qa')
    
    output = {'corp_participants': corp_participants,
              'corp_participants_colapsed': corp_participants_colapsed,
              'conf_participants': conf_participants,
              'conf_participants_colapsed': conf_participants_colapsed,
              'presentation': presentation,
              'qa': qa}
    return output

In [630]:
extract_info_from_conference_call_part(conference_call_sep_raw['qa'], corp_participants_colapsed, conf_participants_colapsed)

In [631]:
extract_info_from_conference_call_sep(conference_call_sep_raw)

{'corp_participants': [],
 'corp_participants_colapsed': [],
 'conf_participants': [],
 'conf_participants_colapsed': [],
 'presentation': [[1,
   'Operator',
   'operator',
   "Welcome to the Noble Drilling Corporation first quarter earnings conference call. During the presentations, all participants will be in the listen-only mode. Afterwards we'll conduct a question and answer session. At that time, if you have a question, press the 1 followed by the 4 on your telephone. This conference call is being recorded Friday April 26, 2002. I would like to turn the conference over to Mr. James Day, Chairman and CEO of Noble Drilling Corporation. Please go ahead, sir."],
  [2,
   'James Day,',
   'unknown',
   "Thank you. Good morning, everyone. With me this morning is Bob Campbell, President of the company, Mark Jackson, Chief Financial Officer, and Julie Robertson, Senior Vice President of Administration. We will go through our normal drill, that is, I'll ask Mark to make comments about the

In [516]:
def extract_info_from_conference_call_body(body):
    conference_call_sep_raw = seperate_conference_call(body)
    output = extract_info_from_conference_call_sep(conference_call_sep_raw)
    return output

In [517]:
test = extract_info_from_conference_call_body(body)

In [518]:
test

{'corp_participants': [],
 'corp_participants_colapsed': [],
 'conf_participants': [],
 'conf_participants_colapsed': [],
 'presentation': [[1,
   'Unidentified',
   'unknown',
   'The Transcript for this event was not produced due to operational issues that occurred in the initial phase of the Transcript product release. Although these issues have been resolved, this event will not be transcribed because the replay audio has expired.']],
 'qa': None}

In [ ]:
test['c']

In [342]:
# ## Q&A
# # Split qa into slides
# split_qa = '--------------------------------------------------------------------------------\r\n'
# qa = conference_call_sep['qa'].split(split_qa)
# qa = [re.sub(' +', ' ', el.strip()) for el in qa if el.strip()]

# # Split qa into speakers and texts
# qa_speakers = [el for el in qa if re.match('.+\[\d+\]', el)]
# qa_texts = [el for el in qa if el not in qa_speakers]

# regex_pattern = r"(.*)\s+\[(\d+)\]$"
# qa_speakers_ordered = [[int(re.search(regex_pattern, el).group(2)), re.search(regex_pattern, el).group(1)] for el in qa_speakers]
# qa_speakers_ordered
# if len(qa_speakers) != len(qa_texts):
#     print('Warning: qa_speakers and qa_texts have different lengths')
# qa_ordered = [[qa_speakers_ordered[i][0], qa_speakers_ordered[i][1], qa_texts[i]] for i in range(len(qa_speakers))]
# qa_ordered

[[1,
  'Operator',
  "At this time, I would like to remind everyone that in order to ask a question, you press Star 1 on your telephone keypad now. We'll pause for just a moment to compile the Q &amp; A roster. The first question is from Lauren Smith of KBW."],
 [2, 'Lauren Smith, KBW', 'Hi, good afternoon.'],
 [3, 'Richard Handler, Jefferies Group, Inc. - Chairman and CEO', 'Hello.'],
 [4,
  'Lauren Smith, KBW',
  "Could you just provide a little more color relative to your comments on the equity trading front and, you know, perhaps guide us toward what you mean by, uhm, the addition of experienced salespeople and such, you know, materially, uhm, added to, you know, the incrementally higher equity revenues so that, in other words, if, you know, assuming that they weren't on board and given, you know, the pretty horrendous market conditions, what a more, you know, quote, unquote normalized run rate might have been?"],
 [5,
  'Richard Handler, Jefferies Group, Inc. - Chairman and CEO',


In [341]:
# presentation_speakers_ordered[1][1] == ', '.join(corp_participants[0])

True

In [325]:
string = 'Richard Handler, Jefferies Group, Inc. - Chairman and CEO [2]'
regex_pattern = r"(.*)\s+\[(\d+)\]$"
match = re.search(regex_pattern, string)
integer_value = int(match.group(2))
integer_value

2

In [54]:
for event, elem in etree.iterparse(file):
    if elem is not None:
        print(elem.tag)

Headline
Body
EventStory
eventTitle
city
companyName
companyTicker
startDate
Event


In [10]:
import xml.etree.ElementTree as ET
mytree = ET.parse('data/2022/1000015_T.xml')
myroot = mytree.getroot()

In [11]:
## attributes
myroot.attrib

{'Id': '1000015',
 'lastUpdate': 'Tuesday, May 3, 2022 at 2:53:00pm GMT',
 'eventTypeId': '1',
 'eventTypeName': 'Earning Conference Call/Presentation'}

In [36]:
mytree.getChildren()

AttributeError: 'ElementTree' object has no attribute 'getChildren'

In [16]:
## whole text
for x in myroot[0]:
        print(x.text)

Edited Transcript of PBCT.OQ^D22 earnings conference call or presentation 21-Jul-05 2:30pm GMT
Q2 2005 Chittenden Earnings Conference Call

BUFFALO May 3, 2022 (Thomson StreetEvents) -- Edited Transcript of People's United Financial Inc earnings conference call or presentation Thursday, July 21, 2005 at 2:30:00pm GMT

TEXT version of Transcript

Corporate Participants
   *  Kirk Walters
      Chittenden Corporation - EVP and Chief Financial Officer and Treasurer and CTC

Conference Call Participants
   *  Beth Messmore
      Merrill Lynch - Analyst
   *  Troy Ward
      A.G. Edwards - Analyst
   *  Lori Hasiner
      FBR - Analyst
   *  Tom Doheny
      Sandler O'Neill - Analyst
   *  Gerard Cassidy
      RBC Capital Markets - Analyst
   *  Faye Elliott-Gurney
      Lehman Brothers - Analyst

Presentation
--------------------------------------------------------------------------------
Operator   [1]
--------------------------------------------------------------------------------
  Good

In [33]:
myroot.findall('startDate')[0].text

'21-Jul-05 2:30pm GMT'

In [31]:
print(
myroot.findall('eventTitle')[0].text,
myroot.findall('startDate')[0].text,
myroot.findall('companyName')[0].text,
myroot.findall('companyTicker')[0].text,
myroot.findall('city')[0].text,
sep="\n")

Q2 2005 Chittenden Earnings Conference Call
21-Jul-05 2:30pm GMT
People's United Financial Inc
PBCT.OQ^D22
BUFFALO
